In [1]:
from pyspark.sql import SparkSession
import os 

# Spark + Iceberg 설정
spark = SparkSession.builder \
    .appName("IcebergBlogDemo") \
    .config("spark.jars", "/home/jovyan/iceberg-spark.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.demo.type", "hadoop") \
    .config("spark.sql.catalog.demo.warehouse", "file:///home/jovyan/data/warehouse") \
    .getOrCreate()

print("✅ Spark + Iceberg 설정 완료!")

# 카탈로그 확인
spark.sql("SHOW CATALOGS").show()

✅ Spark + Iceberg 설정 완료!
+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [2]:
# =============================================================================
# ⚖️ COW vs MOR 완전 비교 분석
# =============================================================================

print("⚖️ COW vs MOR 방식 완전 비교 분석")
print("=" * 50)

# 기존 COW 테이블과 새 MOR 테이블 비교
cow_table_path = "/home/jovyan/data/warehouse/blog/orders"
mor_table_path = "/home/jovyan/data/warehouse/blog/orders_mor"

# 1. 파일 구조 비교
import glob
import os

print("📁 파일 구조 비교:")
print("=" * 30)

# COW 테이블 분석
cow_all_files = glob.glob(f"{cow_table_path}/**/*.parquet", recursive=True)
cow_data_files = [f for f in cow_all_files if 'delete' not in os.path.basename(f).lower()]
cow_delete_files = [f for f in cow_all_files if 'delete' in os.path.basename(f).lower()]

# MOR 테이블 분석
mor_all_files = glob.glob(f"{mor_table_path}/**/*.parquet", recursive=True)
mor_data_files = [f for f in mor_all_files if 'delete' not in os.path.basename(f).lower()]
mor_delete_files = [f for f in mor_all_files if 'delete' in os.path.basename(f).lower()]

print(f"📊 COW 테이블 (orders):")
print(f"  총 파일: {len(cow_all_files)}개")
print(f"  📄 데이터 파일: {len(cow_data_files)}개")
print(f"  🗑️ Delete 파일: {len(cow_delete_files)}개")

print(f"\n📊 MOR 테이블 (orders_mor):")
print(f"  총 파일: {len(mor_all_files)}개")
print(f"  📄 데이터 파일: {len(mor_data_files)}개")
print(f"  🗑️ Delete 파일: {len(mor_delete_files)}개")

# 2. 스토리지 사용량 비교
cow_total_size = sum(os.path.getsize(f) for f in cow_all_files)
mor_total_size = sum(os.path.getsize(f) for f in mor_all_files)

print(f"\n💾 스토리지 사용량 비교:")
print(f"  COW: {cow_total_size:,} bytes ({cow_total_size/1024/1024:.2f} MB)")
print(f"  MOR: {mor_total_size:,} bytes ({mor_total_size/1024/1024:.2f} MB)")
print(f"  차이: {mor_total_size - cow_total_size:+,} bytes")

# 3. 메타데이터 복잡도 비교
cow_metadata_files = glob.glob(f"{cow_table_path}/metadata/*.avro")
mor_metadata_files = glob.glob(f"{mor_table_path}/metadata/*.avro")

print(f"\n🗂️ 메타데이터 복잡도:")
print(f"  COW Manifest/List 파일: {len(cow_metadata_files)}개")
print(f"  MOR Manifest/List 파일: {len(mor_metadata_files)}개")

# 4. 스냅샷 개수 비교
cow_snapshots = spark.sql("SELECT COUNT(*) FROM demo.blog.orders.snapshots").collect()[0][0]
mor_snapshots = spark.sql("SELECT COUNT(*) FROM demo.blog.orders_mor.snapshots").collect()[0][0]

print(f"\n📸 스냅샷 개수:")
print(f"  COW: {cow_snapshots}개")
print(f"  MOR: {mor_snapshots}개")
print(f"  👉 MOR이 더 많은 이유: 삭제/업데이트 시 새 스냅샷 생성")

# 5. 쿼리 성능 비교 시뮬레이션
print(f"\n🚀 쿼리 성능 특성 비교:")
print("=" * 30)

print("📖 읽기 성능:")
print("  COW: ⭐⭐⭐⭐⭐ (Delete Files 없음, 빠른 스캔)")
print("  MOR: ⭐⭐⭐⭐☆ (Delete Files 병합 필요)")

print("\n✏️ 쓰기 성능:")
print("  COW: ⭐⭐⭐☆☆ (전체 파일 재작성)")
print("  MOR: ⭐⭐⭐⭐⭐ (Delete Files만 생성)")

print("\n🔄 업데이트/삭제 성능:")
print("  COW: ⭐⭐☆☆☆ (전체 파일 재작성 필요)")
print("  MOR: ⭐⭐⭐⭐⭐ (Delete Files로 빠른 처리)")

# 6. 실제 삭제 작업 시간 비교
import time

print(f"\n⏱️ 실제 삭제 작업 시간 비교:")
print("=" * 30)

# COW 삭제 시간 측정
print("🔄 COW 테이블에서 삭제 작업...")
start_time = time.time()
spark.sql("DELETE FROM demo.blog.orders WHERE order_id = 999")  # 존재하지 않는 ID
cow_delete_time = time.time() - start_time

# MOR 삭제 시간 측정  
print("🔄 MOR 테이블에서 삭제 작업...")
start_time = time.time()
spark.sql("DELETE FROM demo.blog.orders_mor WHERE order_id = 1000")  # 존재하지 않는 ID
mor_delete_time = time.time() - start_time

print(f"  COW 삭제 시간: {cow_delete_time:.2f}초")
print(f"  MOR 삭제 시간: {mor_delete_time:.2f}초")
print(f"  성능 차이: {((cow_delete_time - mor_delete_time) / cow_delete_time * 100):+.1f}%")

# 7. 사용 권장 사항
print(f"\n💡 사용 권장 사항:")
print("=" * 20)
print("🎯 COW 방식 추천:")
print("  ✅ 읽기 중심 워크로드")
print("  ✅ 업데이트/삭제가 거의 없음")
print("  ✅ 배치 처리 위주")
print("  ✅ 쿼리 성능이 최우선")

print("\n🎯 MOR 방식 추천:")
print("  ✅ 실시간 업데이트/삭제 필요")
print("  ✅ 스트리밍 데이터")
print("  ✅ 빈번한 변경 작업")
print("  ✅ 쓰기 성능이 중요")

⚖️ COW vs MOR 방식 완전 비교 분석
📁 파일 구조 비교:
📊 COW 테이블 (orders):
  총 파일: 66개
  📄 데이터 파일: 66개
  🗑️ Delete 파일: 0개

📊 MOR 테이블 (orders_mor):
  총 파일: 101개
  📄 데이터 파일: 72개
  🗑️ Delete 파일: 29개

💾 스토리지 사용량 비교:
  COW: 121,883 bytes (0.12 MB)
  MOR: 178,872 bytes (0.17 MB)
  차이: +56,989 bytes

🗂️ 메타데이터 복잡도:
  COW Manifest/List 파일: 11개
  MOR Manifest/List 파일: 11개

📸 스냅샷 개수:
  COW: 6개
  MOR: 5개
  👉 MOR이 더 많은 이유: 삭제/업데이트 시 새 스냅샷 생성

🚀 쿼리 성능 특성 비교:
📖 읽기 성능:
  COW: ⭐⭐⭐⭐⭐ (Delete Files 없음, 빠른 스캔)
  MOR: ⭐⭐⭐⭐☆ (Delete Files 병합 필요)

✏️ 쓰기 성능:
  COW: ⭐⭐⭐☆☆ (전체 파일 재작성)
  MOR: ⭐⭐⭐⭐⭐ (Delete Files만 생성)

🔄 업데이트/삭제 성능:
  COW: ⭐⭐☆☆☆ (전체 파일 재작성 필요)
  MOR: ⭐⭐⭐⭐⭐ (Delete Files로 빠른 처리)

⏱️ 실제 삭제 작업 시간 비교:
🔄 COW 테이블에서 삭제 작업...
🔄 MOR 테이블에서 삭제 작업...
  COW 삭제 시간: 0.65초
  MOR 삭제 시간: 0.47초
  성능 차이: +27.7%

💡 사용 권장 사항:
🎯 COW 방식 추천:
  ✅ 읽기 중심 워크로드
  ✅ 업데이트/삭제가 거의 없음
  ✅ 배치 처리 위주
  ✅ 쿼리 성능이 최우선

🎯 MOR 방식 추천:
  ✅ 실시간 업데이트/삭제 필요
  ✅ 스트리밍 데이터
  ✅ 빈번한 변경 작업
  ✅ 쓰기 성능이 중요


In [3]:
# =============================================================================
# 🔄 Schema Evolution 완전 실습
# =============================================================================

print("🔄 Schema Evolution 완전 실습")
print("=" * 50)

# 1. 현재 스키마 확인
print("📋 현재 MOR 테이블 스키마:")
spark.sql("DESCRIBE demo.blog.orders_mor").show()

current_schema = spark.sql("SHOW CREATE TABLE demo.blog.orders_mor").collect()[0][0]
print(f"현재 CREATE TABLE 문:\n{current_schema}")

# 2. 컬럼 추가 (가장 일반적인 진화)
print(f"\n➕ Step 1: 새 컬럼 추가")
print("=" * 20)

# 새 컬럼들 추가
spark.sql("""
    ALTER TABLE demo.blog.orders_mor 
    ADD COLUMNS (
        discount_rate DECIMAL(5,2) COMMENT '할인율',
        shipping_address STRING COMMENT '배송 주소',
        created_at TIMESTAMP COMMENT '생성 시간'
    )
""")

print("✅ 새 컬럼 3개 추가 완료!")

# 스키마 변경 확인
print("\n📋 변경된 스키마:")
spark.sql("DESCRIBE demo.blog.orders_mor").show()

# 3. 새 컬럼에 데이터 삽입
print(f"\n📝 Step 2: 새 스키마로 데이터 삽입")
print("=" * 30)

spark.sql("""
    INSERT INTO demo.blog.orders_mor VALUES 
    (2001, 500, 'New iPhone 15', DATE('2023-04-15'), 1199.99, 'new', 
     0.10, '서울시 강남구', CURRENT_TIMESTAMP()),
    (2002, 501, 'MacBook Air M3', DATE('2023-04-16'), 1599.99, 'processing',
     0.05, '부산시 해운대구', CURRENT_TIMESTAMP())
""")

print("✅ 새 스키마로 데이터 삽입 완료!")

# 4. 기존 데이터와 새 데이터 호환성 확인
print(f"\n🔍 Step 3: 스키마 호환성 확인")
print("=" * 30)

print("📊 전체 데이터 (NULL 처리 확인):")
spark.sql("""
    SELECT order_id, product_name, discount_rate, shipping_address, created_at
    FROM demo.blog.orders_mor 
    ORDER BY order_id DESC 
    LIMIT 10
""").show(truncate=False)

# 새 컬럼의 NULL 비율 확인
null_stats = spark.sql("""
    SELECT 
        COUNT(*) as total_records,
        COUNT(discount_rate) as discount_rate_non_null,
        COUNT(shipping_address) as shipping_address_non_null,
        COUNT(created_at) as created_at_non_null,
        ROUND(COUNT(discount_rate) * 100.0 / COUNT(*), 2) as discount_rate_fill_rate
    FROM demo.blog.orders_mor
""").collect()[0]

print(f"\n📈 새 컬럼 채움율:")
print(f"  총 레코드: {null_stats['total_records']}")
print(f"  discount_rate 채움율: {null_stats['discount_rate_fill_rate']}%")

# 5. 컬럼 이름 변경
print(f"\n🔄 Step 4: 컬럼 이름 변경")
print("=" * 20)

spark.sql("""
    ALTER TABLE demo.blog.orders_mor 
    RENAME COLUMN shipping_address TO delivery_address
""")

print("✅ shipping_address → delivery_address 변경 완료!")

# 6. 컬럼 타입 진화 (호환 가능한 타입만)
print(f"\n🔄 Step 5: 컬럼 타입 확장")
print("=" * 20)

# 예: DECIMAL(10,2) → DECIMAL(12,2) (더 큰 정밀도)
spark.sql("""
    ALTER TABLE demo.blog.orders_mor 
    ALTER COLUMN amount TYPE DECIMAL(12,2)
""")

print("✅ amount 컬럼 타입 확장 완료!")

# 7. 스키마 히스토리 확인
print(f"\n📚 Step 6: 스키마 히스토리 확인")
print("=" * 30)

# 테이블의 메타데이터에서 스키마 진화 확인
metadata_path = f"{mor_table_path}/metadata"
metadata_files = [f for f in os.listdir(metadata_path) if f.endswith('.metadata.json')]
latest_metadata_file = max(metadata_files, key=lambda x: os.path.getctime(os.path.join(metadata_path, x)))

import json
with open(os.path.join(metadata_path, latest_metadata_file), 'r') as f:
    metadata = json.load(f)

print(f"📋 스키마 진화 히스토리:")
for i, schema in enumerate(metadata['schemas']):
    schema_id = schema['schema-id']
    field_count = len(schema['fields'])
    is_current = schema_id == metadata['current-schema-id']
    
    print(f"  스키마 #{schema_id} ({'현재' if is_current else '이전'}):")
    print(f"    컬럼 수: {field_count}개")
    print(f"    컬럼들: {[f['name'] for f in schema['fields']]}")

# 8. Time Travel로 스키마 변경 전 데이터 조회
print(f"\n⏰ Step 7: 스키마 변경 전 Time Travel")
print("=" * 30)

# 첫 번째 스냅샷으로 돌아가기 (스키마 변경 전)
snapshots = spark.sql("SELECT * FROM demo.blog.orders_mor.snapshots ORDER BY committed_at").collect()
if len(snapshots) > 0:
    first_snapshot = snapshots[0]['snapshot_id']
    
    print(f"📸 첫 번째 스냅샷 (ID: {first_snapshot}) 데이터:")
    try:
        spark.sql(f"""
            SELECT order_id, product_name, amount, status
            FROM demo.blog.orders_mor 
            VERSION AS OF {first_snapshot}
            ORDER BY order_id 
            LIMIT 5
        """).show()
        
        print("👉 새 컬럼들은 존재하지 않음 (스키마 진화 전)")
    except Exception as e:
        print(f"조회 실패: {e}")

# 9. 스키마 진화 모범 사례
print(f"\n💡 Schema Evolution 모범 사례:")
print("=" * 40)
print("✅ 권장 사항:")
print("  🔹 항상 컬럼 추가부터 시작")
print("  🔹 NULL 허용으로 기존 데이터 호환성 유지")
print("  🔹 타입 확장은 호환 가능한 방향으로만")
print("  🔹 컬럼 삭제는 신중하게 (읽기 스키마 활용)")
print("  🔹 변경 전후 데이터 검증 필수")

print("\n❌ 주의 사항:")
print("  🔸 호환되지 않는 타입 변경 금지")
print("  🔸 기존 컬럼 삭제 시 Time Travel 고려")
print("  🔸 대용량 테이블에서는 성능 영향 검토")

🔄 Schema Evolution 완전 실습
📋 현재 MOR 테이블 스키마:
+--------------------+-------------+-------+
|            col_name|    data_type|comment|
+--------------------+-------------+-------+
|            order_id|       bigint|   null|
|         customer_id|       bigint|   null|
|        product_name|       string|   null|
|          order_date|         date|   null|
|              amount|decimal(10,2)|   null|
|              status|       string|   null|
|# Partition Infor...|             |       |
|          # col_name|    data_type|comment|
|          order_date|         date|   null|
+--------------------+-------------+-------+

현재 CREATE TABLE 문:
CREATE TABLE demo.blog.orders_mor (
  order_id BIGINT,
  customer_id BIGINT,
  product_name STRING,
  order_date DATE,
  amount DECIMAL(10,2),
  status STRING)
USING iceberg
PARTITIONED BY (order_date)
LOCATION 'file:///home/jovyan/data/warehouse/blog/orders_mor'
TBLPROPERTIES (
  'current-snapshot-id' = '2226134241659144142',
  'format' = 'iceberg/p

In [4]:
# =============================================================================
# 🗜️ Compaction (파일 최적화) 완전 실습
# =============================================================================

print("🗜️ Compaction (파일 최적화) 완전 실습")
print("=" * 50)

# 1. Compaction 전 상태 분석
print("📊 Compaction 전 파일 상태 분석:")
print("=" * 30)

# 현재 파일 상태 확인
current_files = glob.glob(f"{mor_table_path}/**/*.parquet", recursive=True)
data_files = [f for f in current_files if 'delete' not in os.path.basename(f).lower()]
delete_files = [f for f in current_files if 'delete' in os.path.basename(f).lower()]

print(f"📄 총 데이터 파일: {len(data_files)}개")
print(f"🗑️ Delete 파일: {len(delete_files)}개")

# 파일 크기 분석
file_sizes = [os.path.getsize(f) for f in data_files]
total_size = sum(file_sizes)
avg_size = total_size / len(file_sizes) if file_sizes else 0

print(f"💾 총 데이터 크기: {total_size:,} bytes ({total_size/1024/1024:.2f} MB)")
print(f"📏 평균 파일 크기: {avg_size:,.0f} bytes ({avg_size/1024:.1f} KB)")
print(f"📏 최소 파일 크기: {min(file_sizes):,} bytes")
print(f"📏 최대 파일 크기: {max(file_sizes):,} bytes")

# 작은 파일들 식별 (100KB 미만)
small_files = [f for f in data_files if os.path.getsize(f) < 100 * 1024]
print(f"🔍 작은 파일 (< 100KB): {len(small_files)}개")

# 2. 현재 테이블 성능 확인
print(f"\n📈 Compaction 전 테이블 성능:")
print("=" * 30)

# 레코드 수와 파일 수 비율
total_records = spark.sql("SELECT COUNT(*) FROM demo.blog.orders_mor").collect()[0][0]
records_per_file = total_records / len(data_files) if data_files else 0

print(f"📊 총 레코드: {total_records:,}개")
print(f"📊 파일당 평균 레코드: {records_per_file:.0f}개")

# 파일별 상세 정보
print(f"\n📋 파일별 상세 정보 (처음 5개):")
for i, file_path in enumerate(data_files[:5]):
    size = os.path.getsize(file_path)
    relative_path = file_path.replace(mor_table_path, "")
    
    # 파일의 레코드 수 확인
    try:
        file_df = spark.read.parquet(file_path)
        record_count = file_df.count()
    except:
        record_count = 0
    
    print(f"  {i+1}. {relative_path}")
    print(f"     크기: {size:,} bytes ({size/1024:.1f} KB)")
    print(f"     레코드: {record_count:,}개")

# 3. Data Files Compaction 실행
print(f"\n🗜️ Data Files Compaction 실행:")
print("=" * 30)

# Compaction 전 시간 기록
import time
compaction_start_time = time.time()

# 실제 Compaction 실행 (Spark 3.x 기준)
try:
    print("🔄 Data files compaction 시작...")
    
    # 방법 1: Spark 프로시저 사용
    spark.sql("""
        CALL demo.system.rewrite_data_files(
            table => 'demo.blog.orders_mor'
        )
    """)
    
    print("✅ Data files compaction 완료!")
    
except Exception as e:
    print(f"❌ Compaction 실패: {e}")
    print("대안 방법 시도...")
    
    # 방법 2: 수동 Compaction (INSERT OVERWRITE)
    try:
        spark.sql("""
            CREATE OR REPLACE TABLE demo.blog.orders_mor_temp 
            USING ICEBERG
            PARTITIONED BY (order_date)
            TBLPROPERTIES (
                'write.delete.mode' = 'merge-on-read',
                'write.update.mode' = 'merge-on-read'
            )
            AS SELECT * FROM demo.blog.orders_mor
        """)
        
        spark.sql("DROP TABLE demo.blog.orders_mor")
        spark.sql("ALTER TABLE demo.blog.orders_mor_temp RENAME TO demo.blog.orders_mor")
        
        print("✅ 수동 Compaction 완료!")
        
    except Exception as e2:
        print(f"❌ 수동 Compaction도 실패: {e2}")

compaction_time = time.time() - compaction_start_time

# 4. Compaction 후 상태 분석
print(f"\n📊 Compaction 후 파일 상태 분석:")
print("=" * 30)

# 새로운 파일 상태 확인
after_files = glob.glob(f"{mor_table_path}/**/*.parquet", recursive=True)
after_data_files = [f for f in after_files if 'delete' not in os.path.basename(f).lower()]
after_delete_files = [f for f in after_files if 'delete' in os.path.basename(f).lower()]

print(f"📄 Compaction 후 데이터 파일: {len(after_data_files)}개")
print(f"🗑️ Delete 파일: {len(after_delete_files)}개")

# 파일 크기 분석
if after_data_files:
    after_file_sizes = [os.path.getsize(f) for f in after_data_files]
    after_total_size = sum(after_file_sizes)
    after_avg_size = after_total_size / len(after_file_sizes)
    
    print(f"💾 총 데이터 크기: {after_total_size:,} bytes ({after_total_size/1024/1024:.2f} MB)")
    print(f"📏 평균 파일 크기: {after_avg_size:,.0f} bytes ({after_avg_size/1024:.1f} KB)")

# 5. Compaction 효과 분석
print(f"\n📈 Compaction 효과 분석:")
print("=" * 30)

file_reduction = len(data_files) - len(after_data_files)
file_reduction_pct = (file_reduction / len(data_files) * 100) if data_files else 0

size_change = after_total_size - total_size if after_data_files else 0
size_change_pct = (size_change / total_size * 100) if total_size > 0 else 0

print(f"📊 파일 수 변화: {len(data_files)} → {len(after_data_files)} ({file_reduction:+d}개, {file_reduction_pct:+.1f}%)")
print(f"💾 크기 변화: {size_change:+,} bytes ({size_change_pct:+.1f}%)")
print(f"⏱️ Compaction 시간: {compaction_time:.2f}초")

if after_data_files:
    avg_size_change = after_avg_size - avg_size
    avg_size_change_pct = (avg_size_change / avg_size * 100) if avg_size > 0 else 0
    print(f"📏 평균 파일 크기 변화: {avg_size_change:+,.0f} bytes ({avg_size_change_pct:+.1f}%)")

# 6. Delete Files Compaction
print(f"\n🗑️ Delete Files Compaction:")
print("=" * 30)

if after_delete_files:
    print(f"현재 Delete 파일: {len(after_delete_files)}개")
    print("🔄 Delete files compaction 시도...")
    
    try:
        # Delete files compaction
        spark.sql("""
            CALL demo.system.rewrite_manifests(
                table => 'demo.blog.orders_mor'
            )
        """)
        print("✅ Delete files compaction 완료!")
        
    except Exception as e:
        print(f"❌ Delete files compaction 실패: {e}")
        print("👉 일부 Spark 버전에서는 지원하지 않을 수 있습니다.")
else:
    print("✅ Delete 파일이 없어 Compaction 불필요")

# 7. 쿼리 성능 테스트
print(f"\n🚀 Compaction 후 성능 테스트:")
print("=" * 30)

# 간단한 쿼리 성능 측정
queries = [
    "SELECT COUNT(*) FROM demo.blog.orders_mor",
    "SELECT * FROM demo.blog.orders_mor WHERE order_date = '2023-01-15'",
    "SELECT product_name, SUM(amount) FROM demo.blog.orders_mor GROUP BY product_name"
]

for i, query in enumerate(queries):
    print(f"\n📊 쿼리 #{i+1}: {query}")
    
    start_time = time.time()
    result = spark.sql(query)
    result.collect()  # 실제 실행
    query_time = time.time() - start_time
    
    print(f"  ⏱️ 실행 시간: {query_time:.3f}초")

# 8. Compaction 모범 사례
print(f"\n💡 Compaction 모범 사례:")
print("=" * 30)
print("✅ 언제 Compaction 해야 하나:")
print("  🔹 작은 파일이 많을 때 (< 100MB)")
print("  🔹 Delete 파일이 많이 쌓였을 때")
print("  🔹 쿼리 성능이 느려졌을 때")
print("  🔹 정기적인 유지보수 작업으로")

print("\n⚙️ Compaction 전략:")
print("  🔹 오프 피크 시간에 실행")
print("  🔹 파티션별로 나누어 실행")
print("  🔹 타겟 파일 크기 설정 (128MB~1GB)")
print("  🔹 Compaction 후 성능 모니터링")

🗜️ Compaction (파일 최적화) 완전 실습
📊 Compaction 전 파일 상태 분석:
📄 총 데이터 파일: 74개
🗑️ Delete 파일: 29개
💾 총 데이터 크기: 137,935 bytes (0.13 MB)
📏 평균 파일 크기: 1,864 bytes (1.8 KB)
📏 최소 파일 크기: 1,753 bytes
📏 최대 파일 크기: 2,821 bytes
🔍 작은 파일 (< 100KB): 74개

📈 Compaction 전 테이블 성능:
📊 총 레코드: 80개
📊 파일당 평균 레코드: 1개

📋 파일별 상세 정보 (처음 5개):
  1. /data/order_date=2023-01-01/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00001.parquet
     크기: 1,834 bytes (1.8 KB)
     레코드: 1개
  2. /data/order_date=2023-01-02/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00002.parquet
     크기: 1,865 bytes (1.8 KB)
     레코드: 2개
  3. /data/order_date=2023-01-03/00000-20-e1625583-dde6-47ea-bd2c-cdeb07d86e69-00001.parquet
     크기: 1,806 bytes (1.8 KB)
     레코드: 1개
  4. /data/order_date=2023-01-03/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00003.parquet
     크기: 1,814 bytes (1.8 KB)
     레코드: 1개
  5. /data/order_date=2023-01-05/00000-5-226e50bc-6f72-465e-be45-604bea069c98-00004.parquet
     크기: 1,835 bytes (1.8 KB)
     레코드: 1개

🗜️ Data Files Compa

In [7]:
import pandas as pd

# 다른 필요한 imports도 함께
from pyspark.sql.functions import col, min as spark_min, max as spark_max
import glob
import os
import time
import statistics

# =============================================================================
# 🏁 Iceberg 성능 벤치마크 완전 테스트
# =============================================================================

print("🏁 Iceberg 성능 벤치마크 완전 테스트")
print("=" * 50)

import time
import statistics

# 1. 테스트 데이터 준비 (더 큰 데이터셋)
print("📝 대용량 테스트 데이터 생성:")
print("=" * 30)

def generate_large_dataset(num_records=1000):
    """더 큰 테스트 데이터셋 생성"""
    import random
    from datetime import datetime, timedelta
    
    products = [
        'iPhone 14', 'iPhone 14 Pro', 'MacBook Pro', 'MacBook Air', 'iPad Air', 
        'iPad Pro', 'AirPods', 'AirPods Pro', 'Apple Watch', 'Magic Mouse',
        'Magic Keyboard', 'iMac', 'Mac Studio', 'Apple TV', 'HomePod'
    ] * 10  # 더 많은 variety
    
    statuses = ['completed', 'pending', 'shipped', 'cancelled', 'processing']
    
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    
    data = []
    for i in range(num_records):
        random_days = random.randint(0, (end_date - start_date).days)
        order_date = start_date + timedelta(days=random_days)
        
        data.append({
            'order_id': i + 10000,  # 기존 데이터와 구분
            'customer_id': random.randint(1000, 9999),
            'product_name': random.choice(products),
            'order_date': order_date.strftime('%Y-%m-%d'),
            'amount': round(random.uniform(50, 3000), 2),
            'status': random.choice(statuses)
        })
    
    return data

# 벤치마크용 새 테이블 생성
print("🔄 벤치마크용 대용량 테이블 생성...")

try:
    spark.sql("DROP TABLE IF EXISTS demo.blog.orders_benchmark")
except:
    pass

spark.sql("""
    CREATE TABLE demo.blog.orders_benchmark (
        order_id BIGINT,
        customer_id BIGINT,
        product_name STRING,
        order_date DATE,
        amount DECIMAL(10,2),
        status STRING
    ) USING ICEBERG
    PARTITIONED BY (order_date)
    TBLPROPERTIES (
        'write.target-file-size-bytes' = '134217728'  -- 128MB
    )
""")

# 대용량 데이터 생성 및 삽입
large_data = generate_large_dataset(1000)  # 1000개 레코드
df_large = spark.createDataFrame(pd.DataFrame(large_data))
df_large = df_large.withColumn("order_date", col("order_date").cast("date"))

start_time = time.time()
df_large.write.format("iceberg").mode("append").saveAsTable("demo.blog.orders_benchmark")
insert_time = time.time() - start_time

print(f"✅ 1000개 레코드 삽입 완료!")
print(f"⏱️ 삽입 시간: {insert_time:.2f}초")

# 2. 읽기 성능 벤치마크
print(f"\n📖 읽기 성능 벤치마크:")
print("=" * 30)

read_queries = [
    ("전체 카운트", "SELECT COUNT(*) FROM demo.blog.orders_benchmark"),
    ("필터링", "SELECT * FROM demo.blog.orders_benchmark WHERE amount > 1000"),
    ("집계", "SELECT product_name, COUNT(*), AVG(amount) FROM demo.blog.orders_benchmark GROUP BY product_name"),
    ("파티션 스캔", "SELECT * FROM demo.blog.orders_benchmark WHERE order_date >= '2023-06-01' AND order_date <= '2023-06-30'"),
    ("조인", """
        SELECT b1.product_name, COUNT(*) 
        FROM demo.blog.orders_benchmark b1 
        JOIN demo.blog.orders_benchmark b2 ON b1.customer_id = b2.customer_id 
        GROUP BY b1.product_name
    """)
]

read_results = {}
for query_name, query in read_queries:
    print(f"\n🔍 {query_name} 테스트:")
    
    # 워밍업 실행
    spark.sql(query).collect()
    
    # 실제 측정 (3회 평균)
    times = []
    for i in range(3):
        start_time = time.time()
        result = spark.sql(query)
        result.collect()
        times.append(time.time() - start_time)
    
    avg_time = statistics.mean(times)
    min_time = min(times)
    max_time = max(times)
    
    read_results[query_name] = avg_time
    
    print(f"  ⏱️ 평균: {avg_time:.3f}초")
    print(f"  ⏱️ 최소: {min_time:.3f}초") 
    print(f"  ⏱️ 최대: {max_time:.3f}초")

# 3. 쓰기 성능 벤치마크
print(f"\n✏️ 쓰기 성능 벤치마크:")
print("=" * 30)

# 배치 크기별 삽입 성능
batch_sizes = [10, 50, 100, 200]
write_results = {}

for batch_size in batch_sizes:
    print(f"\n📝 배치 크기 {batch_size}개 삽입 테스트:")
    
    test_data = generate_large_dataset(batch_size)
    df_test = spark.createDataFrame(pd.DataFrame(test_data))
    df_test = df_test.withColumn("order_date", col("order_date").cast("date"))
    
    start_time = time.time()
    df_test.write.format("iceberg").mode("append").saveAsTable("demo.blog.orders_benchmark")
    write_time = time.time() - start_time
    
    records_per_second = batch_size / write_time
    write_results[batch_size] = write_time
    
    print(f"  ⏱️ 시간: {write_time:.3f}초")
    print(f"  🚀 처리율: {records_per_second:.1f} records/sec")

# 4. 업데이트/삭제 성능 벤치마크
print(f"\n🔄 업데이트/삭제 성능 벤치마크:")
print("=" * 30)

# 삭제 성능
print("🗑️ 삭제 성능 테스트:")
delete_queries = [
    ("소량 삭제", "DELETE FROM demo.blog.orders_benchmark WHERE order_id BETWEEN 10000 AND 10010"),
    ("조건부 삭제", "DELETE FROM demo.blog.orders_benchmark WHERE amount < 100"),
    ("파티션 삭제", "DELETE FROM demo.blog.orders_benchmark WHERE order_date = '2023-01-01'")
]

delete_results = {}
for delete_name, delete_query in delete_queries:
    print(f"\n  {delete_name}:")
    
    start_time = time.time()
    spark.sql(delete_query)
    delete_time = time.time() - start_time
    
    delete_results[delete_name] = delete_time
    print(f"    ⏱️ 시간: {delete_time:.3f}초")

# 업데이트 성능
print(f"\n📝 업데이트 성능 테스트:")
update_queries = [
    ("상태 업데이트", "UPDATE demo.blog.orders_benchmark SET status = 'updated' WHERE customer_id BETWEEN 1000 AND 1010"),
    ("가격 업데이트", "UPDATE demo.blog.orders_benchmark SET amount = amount * 1.1 WHERE product_name = 'iPhone 14'")
]

update_results = {}
for update_name, update_query in update_queries:
    print(f"\n  {update_name}:")
    
    start_time = time.time()
    spark.sql(update_query)
    update_time = time.time() - start_time
    
    update_results[update_name] = update_time
    print(f"    ⏱️ 시간: {update_time:.3f}초")

# 5. Time Travel 성능
print(f"\n⏰ Time Travel 성능 벤치마크:")
print("=" * 30)

snapshots = spark.sql("SELECT snapshot_id FROM demo.blog.orders_benchmark.snapshots ORDER BY committed_at").collect()

if len(snapshots) >= 2:
    old_snapshot = snapshots[0]['snapshot_id']
    
    # 현재 데이터 조회
    start_time = time.time()
    current_result = spark.sql("SELECT COUNT(*) FROM demo.blog.orders_benchmark").collect()
    current_time = time.time() - start_time
    
    # Time Travel 조회
    start_time = time.time()
    tt_result = spark.sql(f"SELECT COUNT(*) FROM demo.blog.orders_benchmark VERSION AS OF {old_snapshot}").collect()
    tt_time = time.time() - start_time
    
    print(f"📊 현재 데이터 조회: {current_time:.3f}초")
    print(f"⏰ Time Travel 조회: {tt_time:.3f}초")
    print(f"📈 Time Travel 오버헤드: {((tt_time - current_time) / current_time * 100):+.1f}%")

# 6. 파일 I/O 분석
print(f"\n💾 파일 I/O 분석:")
print("=" * 20)

benchmark_table_path = "/home/jovyan/data/warehouse/blog/orders_benchmark"
benchmark_files = glob.glob(f"{benchmark_table_path}/**/*.parquet", recursive=True)

total_files = len(benchmark_files)
total_size = sum(os.path.getsize(f) for f in benchmark_files)
avg_file_size = total_size / total_files if total_files > 0 else 0

print(f"📄 총 파일 수: {total_files}")
print(f"💾 총 크기: {total_size:,} bytes ({total_size/1024/1024:.2f} MB)")
print(f"📏 평균 파일 크기: {avg_file_size:,.0f} bytes ({avg_file_size/1024:.1f} KB)")

# 7. 성능 요약 리포트
print(f"\n📊 성능 벤치마크 요약 리포트:")
print("=" * 50)

print(f"🏆 최고 성능 항목:")
best_read = min(read_results.items(), key=lambda x: x[1])
best_write = min(write_results.items(), key=lambda x: x[1])

print(f"  📖 읽기: {best_read[0]} ({best_read[1]:.3f}초)")
print(f"  ✏️ 쓰기: 배치 크기 {best_write[0]} ({best_write[1]:.3f}초)")

print(f"\n📈 성능 특성:")
print(f"  📊 평균 읽기 성능: {statistics.mean(read_results.values()):.3f}초")
print(f"  📊 평균 쓰기 성능: {statistics.mean(write_results.values()):.3f}초")

# 파일당 처리 성능
records_per_file = 1000 / total_files if total_files > 0 else 0
print(f"  📄 파일당 평균 레코드: {records_per_file:.0f}개")

print(f"\n💡 성능 최적화 권장사항:")
print("  🔹 파일 크기: 128MB~1GB 유지")
print("  🔹 파티셔닝: 쿼리 패턴에 맞게 설계")
print("  🔹 정기적 Compaction 수행")
print("  🔹 적절한 배치 크기로 쓰기")
print("  🔹 필요시 Z-ordering 고려")

print("\n🎉 모든 벤치마크 테스트 완료!")

🏁 Iceberg 성능 벤치마크 완전 테스트
📝 대용량 테스트 데이터 생성:
🔄 벤치마크용 대용량 테이블 생성...
✅ 1000개 레코드 삽입 완료!
⏱️ 삽입 시간: 9.46초

📖 읽기 성능 벤치마크:

🔍 전체 카운트 테스트:
  ⏱️ 평균: 0.054초
  ⏱️ 최소: 0.048초
  ⏱️ 최대: 0.064초

🔍 필터링 테스트:
  ⏱️ 평균: 0.858초
  ⏱️ 최소: 0.634초
  ⏱️ 최대: 1.284초

🔍 집계 테스트:
  ⏱️ 평균: 0.632초
  ⏱️ 최소: 0.523초
  ⏱️ 최대: 0.805초

🔍 파티션 스캔 테스트:
  ⏱️ 평균: 0.171초
  ⏱️ 최소: 0.069초
  ⏱️ 최대: 0.347초

🔍 조인 테스트:
  ⏱️ 평균: 0.805초
  ⏱️ 최소: 0.624초
  ⏱️ 최대: 1.040초

✏️ 쓰기 성능 벤치마크:

📝 배치 크기 10개 삽입 테스트:
  ⏱️ 시간: 0.679초
  🚀 처리율: 14.7 records/sec

📝 배치 크기 50개 삽입 테스트:
  ⏱️ 시간: 0.958초
  🚀 처리율: 52.2 records/sec

📝 배치 크기 100개 삽입 테스트:
  ⏱️ 시간: 1.421초
  🚀 처리율: 70.4 records/sec

📝 배치 크기 200개 삽입 테스트:
  ⏱️ 시간: 3.097초
  🚀 처리율: 64.6 records/sec

🔄 업데이트/삭제 성능 벤치마크:
🗑️ 삭제 성능 테스트:

  소량 삭제:
    ⏱️ 시간: 1.115초

  조건부 삭제:
    ⏱️ 시간: 0.733초

  파티션 삭제:
    ⏱️ 시간: 0.181초

📝 업데이트 성능 테스트:

  상태 업데이트:
    ⏱️ 시간: 0.427초

  가격 업데이트:
    ⏱️ 시간: 2.161초

⏰ Time Travel 성능 벤치마크:
📊 현재 데이터 조회: 0.086초
⏰ Time Travel 조회: 0.078초
📈 Time Travel 오버헤드: -8.9%

💾 파일 I/O 분석:
📄 총 파일